In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [3]:
input_file_path = 'proustr.pdf'
loader = PyPDFLoader(input_file_path)
texts = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50) # need an optimal chunk size here.
texts = text_splitter.split_documents(texts)

In [1]:
from langchain_community.embeddings import GPT4AllEmbeddings
embedding=GPT4AllEmbeddings()


/Users/atufasheen/Downloads/projects/cg_project1/cg_venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
100%|██████████| 45.9M/45.9M [00:07<00:00, 6.06MiB/s]


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [4]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=texts,embedding=embedding )

In [5]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [11]:
from langchain_community.llms import GPT4All

gpt4all = GPT4All(
    model="/Users/atufasheen/Downloads/projects/cg_project1/local_models/mistral-7b-instruct-v0.1.Q4_0.gguf",
    max_tokens=2048,
)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Chain
def format_docs(docs):
    return docs #"\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | gpt4all | StrOutputParser()

# Run
question = "What are the approaches describd?"
docs = vectorstore.similarity_search(question)


'\n\nThe main themes in these retrieved documents are related to text processing and natural language processing techniques for the French novel "In Search of Lost Time" by Marcel Proust. The documents contain commands and functions for tasks such as detecting days and months, stemming words and sentences, removing punctuation, converting text to lowercase, and keeping only alphanumeric characters. Additionally, there are references to specific characters and books in the novel, suggesting that these processing techniques may be used to analyze or extract information from the text.'

In [ ]:
res = chain.invoke(docs)

In [13]:
docs

[Document(page_content='pr_detect_pro . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9\npr_keep_only_alnum . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10\npr_normalize_punc . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11', metadata={'page': 1, 'source': 'proustr.pdf'}),
 Document(page_content='pr_stem_sentences . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11\npr_stem_words . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 12\npr_unacent . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 13', metadata={'page': 1, 'source': 'proustr.pdf'}),
 Document(page_content='ducotedechezswann , 3\nlaprisonniere , 4\nlecotedeguermantes , 4\nletempretrouve , 5\npr_detect_days , 8\npr_detect_months , 9\npr_detect_pro , 9\npr_keep_only_alnum , 10\npr_normalize_punc , 11\npr_stem_sentences , 11\npr_stem_words , 12\npr_unacent 